In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm
import itertools
import time
import random
import os
from PIL import Image
import csv

In [18]:
class CNN(nn.Module):
    def __init__(self, num_filters, kernel_size, dropout_rate, num_units1, num_units2):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, num_filters, kernel_size=kernel_size, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        
        # Assuming input size is (3, 64, 64)
        self.input_size = 64
        self.conv_output_size = self._get_conv_output_size(self.input_size, kernel_size, 2)  # 2 is the pool size

        self.fc1 = nn.Linear(num_filters * self.conv_output_size * self.conv_output_size, num_units1)
        self.fc2 = nn.Linear(num_units1, num_units2)
        self.fc3 = nn.Linear(num_units2, 5)  # 5 classes for airfield, bus stand, canyon, market, temple
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.view(-1, self.num_flat_features(x))
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

    def _get_conv_output_size(self, input_size, kernel_size, pool_size):
        conv_output_size = (input_size - (kernel_size - 1) - 1 + 2) // 1 + 1
        conv_output_size = (conv_output_size - (pool_size - 1) - 1) // pool_size + 1
        return conv_output_size


In [19]:
def load_data(root_dir, batch_size):
    transform = transforms.Compose([
        transforms.Resize((64, 64)),  # Resize images to a consistent size
        transforms.ToTensor(),        # Convert images to PyTorch tensors
        transforms.Normalize((0.5,), (0.5,))  # Normalize the images
    ])

    # Load datasets using ImageFolder
    train_dataset = datasets.ImageFolder(root=os.path.join(root_dir, 'train'), transform=transform)
    test_dataset = datasets.ImageFolder(root=os.path.join(root_dir, 'test'), transform=transform)
    validation_dataset = datasets.ImageFolder(root=os.path.join(root_dir, 'validation'), transform=transform)

    # Create data loaders for train, test, and validation datasets
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader, validation_loader, train_dataset.classes

In [20]:
# Define the training function
def train_model(model, train_loader, criterion, optimizer, device, num_epochs):
    model.train()  # Set the model to training mode
    for epoch in range(num_epochs):
        running_loss = 0.0
        # Initialize tqdm for the training loop
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()  # Clear the gradients
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Compute the loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update the weights
            running_loss += loss.item()
            # Update tqdm with the loss
            progress_bar.set_postfix(loss=running_loss / len(train_loader))
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}')
    return model

In [21]:
def evaluate_model(model, data_loader, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # Disable gradient computation
        for inputs, labels in tqdm(data_loader, desc='Evaluating', leave=False):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)  # Forward pass
            _, predicted = torch.max(outputs.data, 1)  # Get the predicted class
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy

In [22]:
# Define the function to predict a single image
def predict_single_image(model, image_path, transform, device, class_names):
    model.eval()  # Set the model to evaluation mode
    image = Image.open(image_path)
    image_tensor = transform(image).unsqueeze(0).to(device)  # Preprocess and add batch dimension
    with torch.no_grad():  # Disable gradient computation
        output = model(image_tensor)  # Forward pass
        _, predicted = torch.max(output, 1)  # Get the predicted class index
    return class_names[predicted.item()]  # Return the predicted class name

In [23]:
# Hyperparameter grid
param_grid = {
    'num_filters': [32, 64],
    'kernel_size': [3, 5],
    'dropout_rate': [0.0, 0.2],
    'num_units1': [32, 64],
    'num_units2': [32, 64],
    'learning_rate': [1e-4, 1e-2],
    'batch_size': [32, 64]
}

param_combinations = list(itertools.product(
    param_grid['num_filters'],
    param_grid['kernel_size'],
    param_grid['dropout_rate'],
    param_grid['num_units1'],
    param_grid['num_units2'],
    param_grid['learning_rate'],
    param_grid['batch_size']
))

In [24]:
random_indexes = random.sample(range(len(param_combinations)), 30)
random_param_combinations = [param_combinations[i] for i in random_indexes]

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [28]:
best_accuracy = 0.0
best_params = None
best_model = None

print("Testing " + str(len(random_param_combinations)) + " combination of hyperparameters.")
combination_num = 1

# Open a CSV file for writing
csv_file = open('hyperparameters_and_validation_accuracies.csv', mode='w', newline='')
csv_writer = csv.writer(csv_file)
# Write the header row
csv_writer.writerow(['Combination', 'Num Filters', 'Kernel Size', 'Dropout Rate', 'Num Units1', 'Num Units2', 'Learning Rate', 'Batch Size', 'Num Epochs', 'Validation Accuracy'])


for params in random_param_combinations:
    print("00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000")
    print("Combination - " + str(combination_num))
    print(params)
    num_filters, kernel_size, dropout_rate, num_units1, num_units2, learning_rate, batch_size = params

    model = CNN(num_filters, kernel_size, dropout_rate, num_units1, num_units2).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Load data and get class names
    root_dir = "D:/Study/COMP 6721/COMP6721-project/Dataset"
    train_loader, test_loader, validation_loader, class_names = load_data(root_dir, batch_size)

    num_epochs = 10
    model = train_model(model, train_loader, criterion, optimizer, device, num_epochs)

    # Evaluate on validation set
    validation_accuracy = evaluate_model(model, validation_loader, device)
    print(f"Params: {params}, Validation Accuracy: {validation_accuracy:.4f}")

    csv_writer.writerow([combination_num, num_filters, kernel_size, dropout_rate, num_units1, num_units2, learning_rate, batch_size, num_epochs, validation_accuracy])
    
    combination_num += 1

    if validation_accuracy > best_accuracy:
        best_accuracy = validation_accuracy
        best_params = params
        best_model = model.state_dict()

# Save the best model's state dictionary to a file
timestamp = time.strftime('%Y%m%d_%H%M%S')
torch.save(best_model, f'best_model_{timestamp}.pth')
print(best_params)
csv_file.close() 

Testing 30 combination of hyperparameters.
00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Combination - 1
(32, 5, 0.2, 64, 64, 0.01, 32)


Epoch 1/10, Loss: 1.6759


Epoch 2/10, Loss: 1.5515


Epoch 3/10, Loss: 1.6090


Epoch 4/10, Loss: 1.6111


...

Epoch 7/10, Loss: 0.6325


Epoch 8/10, Loss: 0.6081


Epoch 9/10, Loss: 0.5864


Epoch 10/10, Loss: 0.5643


Params: (32, 5, 0.0, 32, 64, 0.0001, 64), Validation Accuracy: 0.7456
(32, 5, 0.0, 64, 32, 0.0001, 32)


In [29]:
# Load the best model and evaluate on the test set
best_model_loaded = CNN(best_params[0], best_params[1], best_params[2], best_params[3], best_params[4]).to(device)
best_model_loaded.load_state_dict(best_model)
test_accuracy = evaluate_model(best_model_loaded, test_loader, device)
print(f"Best Parameters: {best_params}, Test Accuracy: {test_accuracy:.4f}")

Best Parameters: (32, 5, 0.0, 64, 32, 0.0001, 32), Test Accuracy: 0.7650


In [31]:
from torchviz import make_dot
def visualize_model(model, input_size=(3, 64, 64)):
    x = torch.randn(1, *input_size).to(next(model.parameters()).device)
    y = model(x)
    make_dot(y, params=dict(list(model.named_parameters()) + [('x', x)])).render("cnn_torchviz3", format="png")

In [32]:
visualize_model(best_model_loaded)

In [30]:
print(model)

CNN(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=30752, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=5, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
)
